In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [73]:
diabete= pd.read_csv("data/diabetes.csv")

In [74]:
diabete["Glucose"]= diabete["Glucose"].replace({0: np.nan}).fillna(diabete["Glucose"].mean())
diabete["BloodPressure"]= diabete["BloodPressure"].replace({0: np.nan}).fillna(diabete["BloodPressure"].mean())
diabete["SkinThickness"]= diabete["SkinThickness"].replace({0: np.nan}).fillna(diabete["SkinThickness"].mean())
diabete["Insulin"]= diabete["Insulin"].replace({0: np.nan}).fillna(diabete["Insulin"].mean())
diabete["BMI"]= diabete["BMI"].replace({0: np.nan}).fillna(diabete["BMI"].mean())

In [75]:
diabete.head()

Pregnancies  Glucose  BloodPressure  SkinThickness     Insulin   BMI  \
0            6    148.0           72.0      35.000000   79.799479  33.6   
1            1     85.0           66.0      29.000000   79.799479  26.6   
2            8    183.0           64.0      20.536458   79.799479  23.3   
3            1     89.0           66.0      23.000000   94.000000  28.1   
4            0    137.0           40.0      35.000000  168.000000  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

In [76]:
features= ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
X= diabete[features]
y= diabete["Outcome"].ravel()
m,n = X.shape
number_of_samples = y.size

In [77]:
def feature_normalization(X):
    """
    Normalizes the features in X. returns a normalized version of X where
    the mean value of each feature is 0 and the standard deviation
    is 1. This is often a good preprocessing step to do when working with
    learning algorithms.
    
    Parameters
    ----------
    X : array_like
        The dataset of shape (m x n).
    
    Returns
    -------
    X_normalized : array_like
        The normalized dataset of shape (m x n).
    mean: array_like
        Mean of every column
    standard_deviation : array_like
        Standard Deviation of every column
    """
    standard_deviation = np.std(X, axis= 0)
    mean = np.mean(X, axis= 0)
    X_normalized = (X - mean)/ standard_deviation
    
    return X_normalized, mean, standard_deviation

In [78]:
normalized_X, mean_X, standard_deviation_X = feature_normalization(X)

In [79]:
# add intercept term to martix X
X = np.concatenate([np.ones((len(X), 1)), X], axis= 1)

In [80]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.
    
    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector 
        or a 2-D matrix. 
    
    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.
    """
    # convert every element to numpy array (matrix, vector or scalar)
    z = np.array(z)
    # create free space as size as input 
    g = np.zeros(z.shape)
    # computing sigmoid function
    g = (1 / (1 + np.exp(-z)))
    
    return g

In [81]:
def cost_function(theta, X, y):
    """
    Compute cost_function for logistic regression. 
    
    Parameters
    ----------
    theta : array_like
        The parameters for logistic regression. This a vector
        of shape (n+1, ).
    
    X : array_like
        The input dataset of shape (m x n+1) where m is the total number
        of data points and n is the number of features. We assume the 
        intercept has already been added to the input.
    
    y : arra_like
        Labels for the input. This is a vector of shape (m, ).
    
    Returns
    -------
    J : float
        The computed value for the cost function. 
    """
    # Initialize some useful values like number_of_samples, J and grad
    number_of_samples = y.size
    J = 0
    grad = np.zeros(theta.shape)
    # hypothesis function
    h = sigmoid(np.dot(X, theta.T))
    # implementation of J
    J = (1/number_of_samples) * (np.sum(np.dot(-y, np.log(h))) - np.dot((1-y), (np.log(1-h))))
    
    return J

In [82]:
def gradient_descent(theta, X, y):
    """
    Compute gradient_descent for logistic regression. 
    
    Parameters
    ----------
    theta : array_like
        The parameters for logistic regression. This a vector
        of shape (n+1, ).
    
    X : array_like
        The input dataset of shape (m x n+1) where m is the total number
        of data points and n is the number of features. We assume the 
        intercept has already been added to the input.
    
    y : arra_like
        Labels for the input. This is a vector of shape (m, ).
    
    Returns
    -------
    grad : array_like
        A vector of shape (n+1, ) which is the gradient of the cost
        function with respect to theta, at the current values of theta.
    """
    # Initialize some useful values like number_of_samples, J and grad
    grad = np.zeros(theta.shape)
    # hypothesis function
    h = sigmoid(np.dot(X, theta.T))
    # implementation of gradient
    grad = (1/number_of_samples) * np.dot((h - y), X)
    
    return grad

In [83]:
from scipy import optimize

In [84]:
# define initial values
options= {"maxiter": 400, "disp": True}
initial_theta = np.zeros(n+1)
# optimize cost_function with method= "TNC". for more information check below link
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-tnc.html#optimize-minimize-tnc
# if you define cost_function and gradient_descent in one function, then you need to change below options as below
# the unified function that contains cost and grad will not need to pass another function on
# jac= on optimize.minimize function and only True value is enough, but when we have separate functions
# cost_function is out main function and jac= gradient_descent work same as past
result= optimize.minimize(cost_function, initial_theta, (X, y), jac= gradient_descent, method= "TNC", options= options)
# put optimizied values in variables
# fun property is minimum value and x is optimized theta
cost_value= result.fun
theta_optimized= result.x

In [85]:
def predict(theta, X):
    """
    Predict whether the label is 0 or 1 using learned logistic regression.
    Computes the predictions for X using a threshold at 0.5 
    (i.e., if sigmoid(theta.T*x) >= 0.5, predict 1)
    
    Parameters
    ----------
    theta : array_like
        Parameters for logistic regression. A vecotor of shape (n+1, ).
    
    X : array_like
        The data to use for computing predictions. The rows is the number 
        of points to compute predictions, and columns is the number of
        features.

    Returns
    -------
    p : array_like
        Predictions and 0 or 1 for each row in X. 
    """
    number_of_samples= X.shape[0] # number of exmaple 
    predicted= np.zeros(number_of_samples)
    
    predicted = np.round(sigmoid(np.dot(X, theta.T))) # round using a threshold at 0.5
    return predicted

In [86]:
def accuracy(y_pred, y):
    """
    check prediction accuracy with comparison between predict and actual value
    if two parameter(predicted and actual value) are equal diff will be 0 and 
    otherwise will be 1. count of nonzero devided by len of diff is errors. 1 minus 
    error is our prediction accuracy
    
    Parameters
    ----------
    y_pred : array_like
        Predicted values that output of predict function.
    
    y : array_like
        Actual value of data, (Labels)
        
    Returns
    -------
    number : float
        model accuracy percentage
    """
    diff = y_pred - y
    return 1.0 - (float(np.count_nonzero(diff)) / len(diff))

In [87]:
# Compute accuracy on our training set
model_predicted = predict(theta_optimized, X)
print("Train Accuracy(Numpy Equal Checking): {:.2f} %".format(np.mean(model_predicted == y) * 100))
# Compute accuracy with XNOR logic
model_predicted = predict(theta_optimized, X)
print("Train Accuracy(XNOR Logic): {:.2f} %".format(np.mean(np.invert(np.logical_xor(model_predicted, y))) * 100))

Train Accuracy(Numpy Equal Checking): 78.12 %
Train Accuracy(XNOR Logic): 78.12 %


## Implementation with SKlearn

In [88]:
from sklearn.linear_model import LogisticRegression 

In [89]:
diabete= pd.read_csv("data/diabetes.csv")

In [90]:
diabete["Glucose"]= diabete["Glucose"].replace({0: np.nan}).fillna(diabete["Glucose"].mean())
diabete["BloodPressure"]= diabete["BloodPressure"].replace({0: np.nan}).fillna(diabete["BloodPressure"].mean())
diabete["SkinThickness"]= diabete["SkinThickness"].replace({0: np.nan}).fillna(diabete["SkinThickness"].mean())
diabete["Insulin"]= diabete["Insulin"].replace({0: np.nan}).fillna(diabete["Insulin"].mean())
diabete["BMI"]= diabete["BMI"].replace({0: np.nan}).fillna(diabete["BMI"].mean())

In [91]:
features= ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
X= diabete[features]
y= diabete["Outcome"].ravel()
m,n = X.shape
number_of_samples = y.size

In [173]:
logistic_regression= LogisticRegression(C= 1, max_iter=500, random_state=42)

In [174]:
logistic_regression.fit(X, y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [175]:
prediction= logistic_regression.predict(X)

In [176]:
accuracy(prediction, y)

0.7799479166666666